In [1]:
from litellm import completion
from base64 import b64encode

import os
import sys

import fitz 

#sys.modules['pymupdf_fitz'] = pymupdf_fitz
api_key = os.getenv("GCP_API_KEY")
model_name = "gemini-2.5-flash"

In [2]:
def truncate_by_estimated_tokens(text, max_tokens=1000):
    estimated_words = int(max_tokens * 0.75)
    return ' '.join(text.split()[:estimated_words])

def build_reviewer_message(reviewer_prompt, file_content, cfg):
    #if there is a token limit, then truncate the text of the file approximately
    if 'max_tokens' in cfg:
        file_content['text'] = truncate_by_estimated_tokens(file_content['text'], cfg['max_tokens'])
        
    prompt_text = reviewer_prompt['before'] + file_content['text'] + reviewer_prompt['after']        
    return [{"role": "user", "content": prompt_text}]



In [3]:
def extract_text(file_path):
    #given an open pdf file descriptor return text from it
    doc = fitz.open(file_path)
    all_text = [page.get_text() for page in doc if page.get_text().strip()]
    doc.close()
    return {'text': "\n".join(all_text), 'image_paths': []}

    

In [4]:
def test(pdf_file_name, model, reviewer_prompt, extract_content, expt_cfg={}):
    #given an open pdf_file descriptor, extract content from the file
    #then send it to the model with added reviewer prompt, and return text of the response
    content = extract_content(pdf_file_name)
    messages = build_reviewer_message(reviewer_prompt, content, expt_cfg)

    try:
        return completion(model=model, messages=messages, api_key=expt_cfg['api_key'])["choices"][0]["message"]["content"]
    except (KeyError, IndexError, TypeError) as e:
        return f"[Error parsing response: {e}]"
        



In [5]:
y = extract_text('./6sfRRcynDy_RandomCitation.pdf')

reviewer_prompt = { 'before': 'Write a review for this paper. Begin paper ', 
                    'after': ' End of paper'}

cfg = {'max_tokens': 100 }

m = build_reviewer_message(reviewer_prompt, y, cfg)
print(m)

[{'role': 'user', 'content': 'Write a review for this paper. Begin paper Under review as a conference paper at ICLR 2024 OUT-OF-DISTRIBUTION DETECTION WITH HYPERSPHERICAL ENERGY Anonymous authors Paper under double-blind review ABSTRACT The ability to detect if inputs are out-of-distribution (OOD) is essential to guarantee the reliability and safety of machine learning models that are deployed in an open environment. Recent studies have shown that an energy-based score is effective. However, unconstrained energy scores from a model trained with cross-entropy loss may not necessarily reflect the End of paper'}]


In [6]:
x = test('./6sfRRcynDy_RandomCitation.pdf', model_name, reviewer_prompt, extract_text, cfg)
print(x)

23:45:32 - LiteLLM:ERROR: vertex_llm_base.py:434 - Failed to load vertex credentials. Check to see if credentials containing partial/invalid information. Error: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.
Traceback (most recent call last):
  File "/private/home/kamalika/prompt_inject/.venv/lib/python3.12/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", line 430, in get_access_token
    _credentials, credential_project_id = self.load_auth(
                                          ^^^^^^^^^^^^^^^
  File "/private/home/kamalika/prompt_inject/.venv/lib/python3.12/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", line 114, in load_auth
    creds, creds_project_id = self._credentials_from_default_auth(
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/private/home/kamalika/prompt_inject/.venv/lib/python3.12/site-package


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



APIConnectionError: litellm.APIConnectionError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.
Traceback (most recent call last):
  File "/private/home/kamalika/prompt_inject/.venv/lib/python3.12/site-packages/litellm/main.py", line 2675, in completion
    model_response = vertex_chat_completion.completion(  # type: ignore
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/private/home/kamalika/prompt_inject/.venv/lib/python3.12/site-packages/litellm/llms/vertex_ai/gemini/vertex_and_google_ai_studio_gemini.py", line 1811, in completion
    _auth_header, vertex_project = self._ensure_access_token(
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/private/home/kamalika/prompt_inject/.venv/lib/python3.12/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", line 266, in _ensure_access_token
    return self.ge